In [1]:
import numpy as np
import pandas as pd

In [2]:
import json

In [3]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, Flatten
# from keras.optimizers import Adam

Using TensorFlow backend.


In [4]:
def load_embedding(filename='embedding.csv'):
    return pd.read_csv(filename, header=None)

In [5]:
emb = load_embedding()

In [6]:
emb.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,-0.187625,0.619178,0.176551,0.700783,-0.195763,-0.539308,0.065364,-0.127384,-0.201672,0.325871,...,0.083290,-1.161312,-0.144524,-1.56650,0.874206,0.885304,-0.313262,-0.152760,0.262431,-0.770821
1,-0.584020,0.390310,0.652820,-0.340300,0.194930,-0.834890,0.119290,-0.572910,-0.568440,0.729890,...,0.285860,-0.052162,-0.508180,-0.63459,0.338890,0.284160,-0.203400,-1.233800,0.467150,0.788580
2,0.257690,0.456290,-0.769740,-0.376790,0.592720,-0.063527,0.205450,-0.573850,-0.290090,-0.136620,...,0.030498,-0.395430,-0.385150,-1.00020,0.087599,-0.310090,-0.346770,-0.314380,0.750040,0.970650
3,-0.039369,1.203600,0.354010,-0.559990,-0.520780,-0.669880,-0.754170,-0.653400,-0.232460,0.586860,...,-0.601410,0.504030,-0.083316,0.20239,0.443000,-0.060769,-0.428070,-0.084135,0.491640,0.085654
4,-0.268200,-0.478700,0.180990,-0.538370,-0.240210,-0.562030,0.209440,0.423580,-0.461470,0.769030,...,0.005399,-0.533440,-0.187060,0.52254,0.243610,0.051387,0.272100,-0.444330,0.019464,0.627820


In [7]:
emb_dim = emb.shape[1]

In [8]:
emb_dim

50

In [9]:
def load_dict(filename):
    with open(filename, 'r') as f:
        return json.load(f)

def load_index_word_map(word2ind_filename='word2ind', ind2word_filename='ind2word'):
    word2ind = load_dict(word2ind_filename)
    ind2word = load_dict(ind2word_filename)
    return word2ind, ind2word

In [10]:
word2ind, ind2word = load_index_word_map()

In [11]:
vocab_size = len(word2ind)

In [12]:
def load_training_samples(filename='train.csv'):
    return pd.read_csv(filename, header=None)

In [13]:
dataset = load_training_samples()

In [14]:
def train_valid_split(x, y, test_size=0.3, random_state=43):
    assert len(x) == len(y), 'Feature and label must have same length.'
    np.random.RandomState(seed=random_state)
    length = len(x)
    choices = list(range(length))
    val_choices = np.random.choice(choices, int(length*test_size), replace=False).tolist()
    train_choices = list(set(choices) - set(val_choices))
    return x[train_choices, :], x[val_choices, :], y[train_choices, :], y[val_choices, :]

In [15]:
train_x, val_x, train_y, val_y = train_valid_split(dataset.values[:, :-1], dataset.values[:, -1:], test_size=0.3, random_state=43)

In [16]:
train_x.shape

(1855374, 20)

In [17]:
train_y.shape

(1855374, 1)

In [18]:
val_x.shape

(795160, 20)

In [19]:
val_y.shape

(795160, 1)

In [20]:
emb.shape

(32916, 50)

In [21]:
input_length = train_x.shape[1]

In [22]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=emb_dim, input_length=input_length, weights=[emb]))
model.add(Dropout(.2))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(Dropout(.2))
model.add(LSTM(100, activation='relu'))
model.add(Dropout(.2))
model.add(Dense(100, activation='relu'))
model.add(Dropout(.2))
# model.add(Flatten())
model.add(Dense(vocab_size, activation='softmax'))

In [23]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 50)            1645800   
_________________________________________________________________
dropout_1 (Dropout)          (None, 20, 50)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 20, 100)           60400     
_________________________________________________________________
dropout_2 (Dropout)          (None, 20, 100)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dropout_3 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
__________

In [24]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [27]:
model.fit(train_x, train_y, validation_data=(val_x, val_y), epochs=10, batch_size=5000)

Train on 1855374 samples, validate on 795160 samples
Epoch 1/10
1855374/1855374 [==============================] - 167s - loss: 4.3847 - acc: 0.2385 - val_loss: 4.2867 - val_acc: 0.2550
Epoch 2/10
1855374/1855374 [==============================] - 166s - loss: 4.3599 - acc: 0.2403 - val_loss: 4.2722 - val_acc: 0.2566
Epoch 3/10
1855374/1855374 [==============================] - 166s - loss: 4.3368 - acc: 0.2416 - val_loss: 4.2510 - val_acc: 0.2586
Epoch 4/10
1855374/1855374 [==============================] - 166s - loss: 4.3168 - acc: 0.2431 - val_loss: 4.2368 - val_acc: 0.2598
Epoch 5/10
1855374/1855374 [==============================] - 169s - loss: 4.2977 - acc: 0.2443 - val_loss: 4.2277 - val_acc: 0.2610
Epoch 6/10
1855374/1855374 [==============================] - 168s - loss: 4.2807 - acc: 0.2456 - val_loss: 4.2106 - val_acc: 0.2619
Epoch 7/10
1855374/1855374 [==============================] - 166s - loss: 4.2661 - acc: 0.2465 - val_loss: 4.1972 - val_acc: 0.2633
Epoch 8/10
18553

In [27]:
model.optimizer.lr = 0.01

In [28]:
model.fit(train_x, train_y, validation_data=(val_x, val_y), epochs=10, batch_size=5000)

Train on 1925374 samples, validate on 825160 samples
Epoch 1/10
1925374/1925374 [==============================] - 136s - loss: 4.4266 - acc: 0.2345 - val_loss: 4.3302 - val_acc: 0.2491
Epoch 2/10
1925374/1925374 [==============================] - 137s - loss: 4.4025 - acc: 0.2360 - val_loss: 4.3159 - val_acc: 0.2507
Epoch 3/10
1925374/1925374 [==============================] - 136s - loss: 4.3803 - acc: 0.2373 - val_loss: 4.2970 - val_acc: 0.2518
Epoch 4/10
1925374/1925374 [==============================] - 137s - loss: 4.3624 - acc: 0.2386 - val_loss: 4.2829 - val_acc: 0.2533
Epoch 5/10
1925374/1925374 [==============================] - 136s - loss: 4.3454 - acc: 0.2396 - val_loss: 4.2711 - val_acc: 0.2543
Epoch 6/10
1925374/1925374 [==============================] - 137s - loss: 4.3298 - acc: 0.2405 - val_loss: 4.2616 - val_acc: 0.2554
Epoch 7/10
1925374/1925374 [==============================] - 137s - loss: 4.3144 - acc: 0.2413 - val_loss: 4.2509 - val_acc: 0.2564
Epoch 8/10
19253

In [30]:
model.optimizer.lr = 0.001

In [31]:
model.fit(train_x, train_y, validation_data=(val_x, val_y), epochs=1, batch_size=5000)

Train on 1925374 samples, validate on 825160 samples
Epoch 1/1
1925374/1925374 [==============================] - 137s - loss: 4.2672 - acc: 0.2445 - val_loss: 4.2199 - val_acc: 0.2597


In [28]:
# save model
model.save('model20171229_2.h5')